In [1]:
from mixed_regression_functions import *
import re

In [2]:
lines = {'4': [], '8' : []}
with open('/datahdd/vmanuel/ELMo/Corpora/40k_dbpedia_sentences.txt', 'r') as inp:
    lines['4'] = inp.readlines()
    lines['4'] = [l.split() for l in lines['4']]
    
with open('/datahdd/vmanuel/ELMo/Corpora/80k_dbpedia_sentences.txt', 'r') as inp:
    lines['8'] = inp.readlines()
    lines['8'] = [l.split() for l in lines['8']]

liness = []
liness = lines['4']
liness.extend(lines['8'])
len(liness)

80000

## Retrieve all URI and preprocess them

In [9]:
sons_of = {}

sons_of['Activity'] = ['Sport'
#                       ,'Game' 
                      ]
sons_of['Agent'] = ['FictionalCharacter']
sons_of['Organisation'] = ['Company'
#                            , 'EducationalInstitution', 'GovernmentAgency', 'PoliticalParty', 'SportsLeague'
                           , 'SportsTeam']
sons_of['Thing'] = [
#     'AnatomicalStructure', 
                    'Award', 'Colour'
                    ,'EthnicGroup', 'Language', 'Plant'
#                     , 'Currency'
                    ,'Weapon']
sons_of['Place'] = ['ArchitecturalStructure', 'Planet']
sons_of['Animal'] = ['Bird'
#                      , 'Crustacean'
#                      , 'Fish', 'Amphibian' 
                     , 'Insect', 'Mammal'
#                      , 'Mollusca', 'Reptile'
                    ]
# sons_of['Person'] = ['Artist', 'Scientist'
#                     ,'Athlete', 'Writer'
#                     ]

In [10]:
types = ['dbo:' + s for sublist in sons_of.values() for s in sublist]

classes = [x.replace('dbo:', '') for x in types]

In [11]:
words = []
i = 0
for t in types:
    w = get_from_class(t)
    words.append(w) 
    clear_output()
    i += 1
    print(round(i/len(types), 2))

1.0


In [12]:
comp_words = {c: [] for c in classes}
regex = re.compile(".*?\((.*?)\)")


for i, w_list in enumerate(words):
    for w in w_list:
        w = re.sub(r'\([^)]*\)', '', w).strip()
        w = w.replace("'", "")
        
        if len(w.split('_')) == 2 and len(w.split('-')) < 2:
            if '' not in w.lower().split('_'):
                comp_words[classes[i]].append(w.lower().split('_'))
        elif len(w.split('-')) == 2 and len(w.split('_')) < 2:
            if '' not in w.lower().split('-'):
                comp_words[classes[i]].append(w.lower().split('-'))    

In [13]:
alpha = re.compile('^[A-z]*$')
word_to_index = set([s for sublists in comp_words.values() for sublist in sublists for s in sublist if re.search(alpha, s) and s != ''])
len(word_to_index)

44359

In [14]:
len(comp_words)

15

In [15]:
word_index = {}
for i, line in enumerate(liness):
    for word in line:
        if word in word_to_index:
            try:
                word_index[word].append(i)
            except:
                word_index[word] = [i]
    if i % 100 == 0:
        clear_output()
        print(i)
word_index = {k:set(v) for k,v in word_index.items()}

79900


In [16]:
comp_word_indexes = {c: [] for c in classes}
for k, typelist in comp_words.items():
    for couple in typelist:
        try:
            inter = word_index[couple[0]].intersection(word_index[couple[1]])
            if inter:
                comp_word_indexes[k].append([couple, list(inter)])
        except:
            pass

In [17]:
real_couples_indexes = {}
tot = len(comp_word_indexes.values())
j = 0
for k, v in comp_word_indexes.items():
    for couple in v:
        for index in couple[1]:
            if abs(liness[index].index(couple[0][0]) - 
                   liness[index].index(couple[0][1])) == 1:
                try:
                    real_couples_indexes["{}_{}".format(couple[0][0], couple[0][1])].append([
                        [liness[index].index(couple[0][0]), liness[index].index(couple[0][1])],
                        index])
                except:
                    real_couples_indexes["{}_{}".format(couple[0][0], couple[0][1])] = [k, 
                                                                                        [
                                                                                            [liness[index].index(couple[0][0]), 
                                                                                             liness[index].index(couple[0][1])],
                                                                                            index]]
    j += 1
    clear_output()
    print(round(j/tot, 2))

1.0


In [18]:
import pickle

def save_data_with_pickle(relative_path, data):
    """ Save data using pickle (serialize) """

    with open(relative_path, 'wb') as handle:
        pickle.dump(data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [19]:
save_data_with_pickle('/datahdd/vmanuel/datasets/composite_words_index', real_couples_indexes)